In [27]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')
# import requests
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
# import pickle
# from sklearn.metrics.pairwise import cosine_similarity
# from langchain_ollama.chat_models import ChatOllama
# from langchain_core.runnables import RunnableLambda
# from langchain.memory import ConversationBufferMemory
# from langchain_ollama.chat_models import ChatOpenAI
# from langchain.schema import Sys
# from langgraph.graph import StateGraph, END

In [24]:
key = open('../../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

base_ = open('../../../../base_url.txt','r')
base_url = base_.read()
# openai.api_key = api_key

# GPT.ver

In [21]:
def invoke(
           prompt,
           api_key=api_key,
           model="gpt-4.1",
           temperature=0):
    client = OpenAI(api_key=api_key)

    # 메시지 구성
    messages = [
        # {"role": "system", "content": prompt},
        {"role": "user", "content": prompt}
    ]
    # 스트리밍 요청
    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        # stream=True  # ⭐ 핵심 옵션!
    )

    # generator 반환 (chunk 단위 텍스트 출력)
    return stream

# Fucntion

In [28]:
class OllamaSentenceTransformer():
    def __init__(
            self,
            *args,
            **kargs,
            ) -> None:
                # self.base_url = kargs.get("base_url", "http://localhost:11434")
                self.model = kargs.get("model","bge-m3")
                self.embedding_model = OllamaEmbeddings(
                            model = self.model,
                            base_url = base_url,
                        )
                        
    
    def encode(
            self,
            documents:Union[str, List[str], np.ndarray],
            *args,
            **kargs,
        )-> np.ndarray:
        if isinstance(documents, str):
            document_embeddings = self.embedding_model.embed_query(documents)
            return np.array(document_embeddings)
        
        if isinstance(documents, np.ndarray):
            documents = documents.tolist()
        
        document_embeddings = [self.embedding_model.embed_query(s) for s in documents]
        return np.array(document_embeddings)
        


In [30]:
# Node 1: 질문 내용 구분 함수
def classify_question(stats):
    # input_question = {"question": "3 곱하기 7은 얼마야?"}

    question = stats["question"]
    messages = [
                    {"role": "system", 
                        "content": f"""당신은 질문이 어떤 분야의 질문을 하는 분류하는 분류기입니다. 
                        수학 문제면 'math', 역사 문제면 'history', 과학문제면 'science', 아니면 'general'로만 대답하세요.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ]
    response = invoke(messages).content.strip().lower()
    stats['type']=response
    return stats

In [14]:
prompt = f"""
                너는 사용자의 질문에 똑똑하고 정확하게 대답해주는 도우미야.
                포카라에 대해서 설명해줘
                """

In [15]:
response = invoke(prompt).choices[0].message.content

In [17]:
print(response)

네, 포카라(Pokhara)에 대해 설명해드릴게요!

**포카라**는 네팔 중서부에 위치한 도시로, 카트만두에 이어 네팔에서 두 번째로 큰 도시입니다. 해발 약 800m에 자리잡고 있으며, 히말라야 산맥의 아름다운 풍경과 자연 경관으로 매우 유명합니다.

### 주요 특징

1. **관광 명소**
   - **페와 호수(Phewa Lake)**: 포카라에서 가장 유명한 호수로, 배를 타고 호수 한가운데에 있는 바라히 사원(Barahi Temple)까지 갈 수 있습니다.
   - **사랑코트(Sarangkot)**: 일출과 히말라야 산맥(특히 마차푸차레와 안나푸르나 봉우리)을 감상할 수 있는 전망대로 유명합니다.
   - **데이비스 폭포(Davis Falls)**: 강물이 지하로 빨려 들어가는 독특한 폭포입니다.
   - **세계 평화 파고다(World Peace Pagoda)**: 멋진 전망과 평화로운 분위기를 즐길 수 있는 불교 사원입니다.

2. **등산과 트레킹의 관문**
   - 포카라는 안나푸르나(Annapurna) 트레킹의 출발점으로, 세계 각지의 트레커들이 모이는 곳입니다.
   - 패러글라이딩, 래프팅, 산악자전거 등 다양한 액티비티도 인기가 많아요.

3. **자연 경관**
   - 히말라야의 눈 덮인 봉우리와 푸른 호수, 울창한 숲이 어우러져 그림 같은 풍경을 자랑합니다.

4. **기후**
   - 포카라는 카트만두보다 온화하고, 연중 대부분 날씨가 좋습니다.

### 요약
포카라는 히말라야 산맥의 아름다운 풍경, 트레킹과 다양한 액티비티, 그리고 평화로운 분위기로 유명한 네팔의 대표적인 관광 도시입니다. 자연을 좋아하거나, 트레킹과 액티비티를 즐기고 싶은 분들에게 꼭 추천하는 여행지입니다!
